In [ ]:
import requests
import os
import uuid
import time
import sys
import pandas as pd
from dotenv import load_dotenv

# Mude para False quando quiser rodar PRA VALER
MODO_TESTE = False 

# O Token tem que ser o mesmo que está configurado no Render
WEBHOOK_TOKEN = load_dotenv("WEBHOOK_TOKEN")
WEBHOOK_URL = f"https://weebhook-btg.onrender.com/webhook?token={WEBHOOK_TOKEN}"

# URL para checar se o servidor está vivo (endpoint leve)
URL_CHECK_SERVER = "https://weebhook-btg.onrender.com/meu-ip"

# Intervalo para respeitar 50 requisicoes por minuto (60s / 50 = 1.2s)
INTERVALO_ENTRE_PEDIDOS = 1.5

# Datas do Relatório
DT_INI = "2026-01-01"
DT_FIM = "2026-01-31"

sys.path.insert(0, r'C:\Scripts\modules_azure\database')
sys.path.insert(0, r'C:\Scripts\modules_azure\parameters')

# Importando os módulos novos
from connection_azure import Connect
from azure_loader import AzureLoader
from parametros import Parametros



In [26]:
def acordar_servidor():
    print("Acordando o servidor no Render...", end=" ")
    try:
        requests.get(URL_CHECK_SERVER, timeout=30)
        print("Servidor Online.")
    except:
        print("Servidor demorou, mas deve ter acordado.")

def get_token(client_id, client_secret):
    url = "https://api.btgpactual.com/iaas-auth/api/v1/authorization/oauth2/accesstoken"
    headers = {
        'x-id-partner-request': str(uuid.uuid4()),
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json'
    }
    data = {'grant_type': 'client_credentials'}
    
    try:
        # VOLTEI PARA O PADRAO DO SEU CODIGO LEGADO
        r = requests.post(url, data=data, headers=headers, auth=(client_id, client_secret))
        
        if r.status_code == 200:
            # AQUI ESTAVA A DIFERENCA: O BTG MANDA NO HEADER, NAO NO JSON
            token = r.headers.get('access_token')
            if token:
                return token
            else:
                # Caso raro: 200 OK mas sem token no header, tenta JSON por desencargo
                try:
                    return r.json().get('access_token')
                except:
                    return None
        else:
            print(f"ERRO AO GERAR TOKEN: Status {r.status_code}")
            return None
    except Exception as e:
        print(f"ERRO DE CONEXAO NO TOKEN: {e}")
        return None

def pedir_relatorio(token, conta, dt_ini, dt_fim):
    url = "https://api.btgpactual.com/iaas-profitability/api/v1/performance-report/account"
    
    headers = {
        'x-id-partner-request': str(uuid.uuid4()),
        'access_token': token,
        'Content-Type': 'application/json'
    }
    
    body = {
        "accountNumber": conta,
        "startDate": dt_ini,
        "endDate": dt_fim,
        "webhookService": "performance-report",
        "callbackUrl": WEBHOOK_URL 
    }
    
    print(f"Disparando conta {conta}...", end=" ")
    
    try:
        r = requests.post(url, headers=headers, json=body, timeout=10)
        
        if r.status_code in [200, 202]:
            print("OK")
        elif r.status_code == 429:
            print("RATE LIMIT. Pausando por 60 segundos...")
            time.sleep(60)
        else:
            print(f"FALHA {r.status_code}: {r.text}")
            
    except Exception as e:
        print(f"ERRO DE REQUEST: {e}")

def carregar_e_filtrar_contas():
    print("Conectando ao banco de dados via AzureLoader...")
    try:
        df_base = AzureLoader.ler_tabela("base_btg") 
        
        if df_base.empty:
            print("Aviso: A tabela retornou vazia.")
            return []

        print(f"Total registros brutos: {len(df_base)}")
        
        coluna_contas = df_base['Conta'].astype(str).str.strip().tolist()
        contas_filtradas = []
        
        for c in coluna_contas:
            if not any(char.isalpha() for char in c) and len(c) > 0:
                contas_filtradas.append(c)
                
        return contas_filtradas
    except Exception as e:
        print(f"Erro ao carregar banco: {e}")
        return []

In [ ]:
if __name__ == "__main__":
    
    # 1. Carrega Credenciais (IGUAL AO SEU LEGADO)
    try:
        load_dotenv()
        # Primeiro tenta minusculo (padrao do seu legado)
        CLIENT_ID = os.getenv("btg_client_id")
        CLIENT_SECRET = os.getenv("btg_client_secret")
        
        # Se nao achar, tenta maiusculo
        if not CLIENT_ID:
            # Tenta encoding utf-16 (comum no Windows/Powershell)
            load_dotenv(encoding='utf-16')
            CLIENT_ID = os.getenv("btg_client_id") or os.getenv("BTG_CLIENT_ID")
            CLIENT_SECRET = os.getenv("btg_client_secret") or os.getenv("BTG_CLIENT_SECRET")
            
        if not CLIENT_ID or not CLIENT_SECRET:
            print("ERRO: Nao encontrei btg_client_id no arquivo .env")
            sys.exit()
            
    except Exception as e:
        print(f"Erro lendo .env: {e}")
        sys.exit()

    # 2. Acorda o Render
    acordar_servidor()
    
    # 3. Carrega Contas
    meus_clientes = carregar_e_filtrar_contas()
    
    if not meus_clientes:
        print("Nenhuma conta valida encontrada. Encerrando.")
        sys.exit()

    # --- MODO TESTE ---
    if MODO_TESTE:
        print("--- MODO DE TESTE ATIVO ---")
        print(f"Webhook configurado para: {WEBHOOK_URL}")
        print(f"Contas carregadas: {len(meus_clientes)}")
        print("Para rodar valendo, mude MODO_TESTE para False.")
        sys.exit()
        
    # --- MODO PRODUCAO ---
    print("Gerando token de acesso...")
    token = get_token(CLIENT_ID, CLIENT_SECRET)
    
    if token:
        print(f"Token gerado com sucesso! (Inicio: {token[:10]}...)")
        print("--- INICIANDO DISPAROS ---")
        print(f"Total: {len(meus_clientes)} contas")
        
        for i, conta in enumerate(meus_clientes, 1):
            print(f"[{i}/{len(meus_clientes)}]", end=" ")
            pedir_relatorio(token, conta, DT_INI, DT_FIM)
            
            # Ping para manter o Render acordado
            if i % 20 == 0:
                try:
                    requests.get(URL_CHECK_SERVER, timeout=1)
                except:
                    pass

            time.sleep(INTERVALO_ENTRE_PEDIDOS)
    else:
        print("Falha critica ao obter Token do BTG.")

Acordando o servidor no Render... Servidor Online.
Conectando ao banco de dados via AzureLoader...
[AzureLoader] Lendo: dbo.base_btg...
Total registros brutos: 1244
Gerando token de acesso...
Token gerado com sucesso! (Inicio: eyJhbGciOi...)
--- INICIANDO DISPAROS ---
Total: 1187 contas
[1/1187] Disparando conta 1003205... OK
[2/1187] Disparando conta 1012749... OK
[3/1187] Disparando conta 1080879... OK
[4/1187] Disparando conta 1090873... OK
[5/1187] Disparando conta 1114763... OK
[6/1187] Disparando conta 11183241... OK
[7/1187] Disparando conta 1127452... OK
[8/1187] Disparando conta 11483666... OK
[9/1187] Disparando conta 1155683... OK
